참조

https://colab.research.google.com/drive/1T8P5DrwBBZpx-FjWrAxXNhZNfsco8y-t?usp=sharing

https://www.kaggle.com/code/carlmcbrideellis/tabnet-a-very-simple-regression-example/notebook

Install

In [1]:
# !pip install tabnet[gpu]
# !pip install tabnet[cpu]
!pip install pytorch-tabnet
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import

In [2]:
import pandas as pd
import numpy  as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tqdm import tqdm

### Prepare the data

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe


In [4]:
train = pd.read_csv("../../00_original_data/FIFA_train.csv")
test = pd.read_csv("../../00_original_data/FIFA_test.csv")

#### Preprocess

In [5]:
# 필요하지 않는 feature 값 제거
train.drop(['id', 'name'], axis = 1, inplace = True)
test.drop(['id', 'name'], axis = 1, inplace = True)

In [6]:
#타겟 데이터
y_train = train['value']

#학습 데이터
x_train = train.copy()

#타겟 생성 후 필요없는 열제거
del x_train['value']

In [7]:
# 카테고리 값 확인
print(x_train['continent'].unique())
print(x_train['position'].unique())
print(x_train['prefer_foot'].unique())

['south america' 'europe' 'africa' 'asia' 'oceania']
['ST' 'GK' 'DF' 'MF']
['left' 'right']


In [8]:
# categorical data를 담고있는 Series는 특수 메서드인 cat 속성이 있다.
# 이를 통해 catagories 와 codes 나 categorical 메서드 등에 쉽게 접근할 수 있다.
# 하지만 시리즈에 바로 categories나 codes를 사용하면 에러가 발생하는데 이때 이용하는 것이 cat 속성이다.

x_train['continent']=x_train['continent'].astype('category').cat.codes
test['continent'] = test['continent'].astype('category').cat.codes

x_train['position'] = x_train['position'].astype('category').cat.codes
test['position'] = test['position'].astype('category').cat.codes

x_train['prefer_foot'] = x_train['prefer_foot'].astype('category').cat.codes
test['prefer_foot'] = test['prefer_foot'].astype('category').cat.codes

In [9]:
# 계약 연도에 월, 일 값이 붙어 있는 데이터를 전처리한 뒤,
# 계약 연도에서 object -> int 로 변환
def func(string:object) -> int:
    string = string[-4:] # 계약연도가 제일 끝에 있거나 단독으로 쓰이고 있어 뒤에서 4번째부터 가지고 옴.
    return int(string)

x_train['contract_until'] = x_train['contract_until'].apply(func)
test['contract_until'] = test['contract_until'].apply(func)

##### Train & Test split

In [10]:
X, X_test, y, y_true = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

X_test_raw = X_test.copy()
X_test = np.array(X_test)

y_true_raw = y_true.copy()
y_true = np.array(y_true).reshape(-1, 1)

### 5 fold TabNet

In [15]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_valid = np.array(X.iloc[train_index]), np.array(X.iloc[test_index])
    y_train, y_valid = np.array(y.iloc[train_index]).reshape(-1, 1), np.array(y.iloc[test_index]).reshape(-1, 1)
    regressor = TabNetRegressor(verbose=0,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=300, max_epochs=2000,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(regressor.predict(X_test))

predictions = np.mean(predictions_array,axis=0)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
1it [08:07, 487.56s/it]

Stop training because you reached max_epochs = 2000 with best_epoch = 1978 and best_val_0_rmse = 5228227.78967


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2it [16:22, 491.64s/it]

Stop training because you reached max_epochs = 2000 with best_epoch = 1969 and best_val_0_rmse = 4211747.72941


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
3it [24:36, 492.70s/it]

Stop training because you reached max_epochs = 2000 with best_epoch = 1997 and best_val_0_rmse = 4999838.33797


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
4it [32:50, 493.38s/it]

Stop training because you reached max_epochs = 2000 with best_epoch = 1984 and best_val_0_rmse = 4817621.86849


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
5it [41:04, 492.87s/it]

Stop training because you reached max_epochs = 2000 with best_epoch = 1969 and best_val_0_rmse = 5368403.2809


In [16]:
print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )

The CV score is 4925167.80129


In [17]:
RMSLE = np.sqrt( mean_squared_error(y_true, predictions) )
print("The LB score is %.5f" % RMSLE )

The LB score is 5947952.58932


### 5 fold TabNet #2

- patience=500
- max_epochs=10000

In [19]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_valid = np.array(X.iloc[train_index]), np.array(X.iloc[test_index])
    y_train, y_valid = np.array(y.iloc[train_index]).reshape(-1, 1), np.array(y.iloc[test_index]).reshape(-1, 1)
    regressor = TabNetRegressor(verbose=0,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=500, max_epochs=10000,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(regressor.predict(X_test))

predictions = np.mean(predictions_array,axis=0)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
1it [23:01, 1381.05s/it]


Early stopping occurred at epoch 5652 with best_epoch = 5152 and best_val_0_rmse = 569297.16156


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2it [42:22, 1252.17s/it]


Early stopping occurred at epoch 4780 with best_epoch = 4280 and best_val_0_rmse = 809658.23059


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
3it [1:07:10, 1359.56s/it]


Early stopping occurred at epoch 6067 with best_epoch = 5567 and best_val_0_rmse = 515720.27993


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
4it [1:34:08, 1461.52s/it]


Early stopping occurred at epoch 6652 with best_epoch = 6152 and best_val_0_rmse = 408447.32735


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
5it [2:01:15, 1455.14s/it]


Early stopping occurred at epoch 6612 with best_epoch = 6112 and best_val_0_rmse = 577908.83874


In [20]:
print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )

The CV score is 576206.36763


In [21]:
RMSLE = np.sqrt( mean_squared_error(y_true, predictions) )
print("The LB score is %.5f" % RMSLE )

The LB score is 1139779.82409


### 5 fold TabNet #3

- patience=400
- max_epochs=6000

In [22]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_valid = np.array(X.iloc[train_index]), np.array(X.iloc[test_index])
    y_train, y_valid = np.array(y.iloc[train_index]).reshape(-1, 1), np.array(y.iloc[test_index]).reshape(-1, 1)
    regressor = TabNetRegressor(verbose=0,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=400, max_epochs=6000,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(regressor.predict(X_test))

predictions = np.mean(predictions_array,axis=0)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
1it [22:53, 1373.94s/it]


Early stopping occurred at epoch 5552 with best_epoch = 5152 and best_val_0_rmse = 569297.16156


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2it [42:12, 1247.32s/it]


Early stopping occurred at epoch 4680 with best_epoch = 4280 and best_val_0_rmse = 809658.23059


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
3it [1:02:57, 1246.13s/it]


Early stopping occurred at epoch 5057 with best_epoch = 4657 and best_val_0_rmse = 618828.69316


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
4it [1:24:00, 1252.97s/it]


Early stopping occurred at epoch 5120 with best_epoch = 4720 and best_val_0_rmse = 529738.02169


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
5it [1:45:13, 1262.77s/it]


Early stopping occurred at epoch 5164 with best_epoch = 4764 and best_val_0_rmse = 920759.65446


In [23]:
print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )

The CV score is 689656.35229


In [24]:
RMSLE = np.sqrt( mean_squared_error(y_true, predictions) )
print("The LB score is %.5f" % RMSLE )

The LB score is 1327738.50707


### 5 fold TabNet #4

- patience=800
- max_epochs=12000

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_valid = np.array(X.iloc[train_index]), np.array(X.iloc[test_index])
    y_train, y_valid = np.array(y.iloc[train_index]).reshape(-1, 1), np.array(y.iloc[test_index]).reshape(-1, 1)
    regressor = TabNetRegressor(verbose=0,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=800, max_epochs=12000,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(regressor.predict(X_test))

predictions = np.mean(predictions_array,axis=0)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
1it [25:31, 1531.14s/it]


Early stopping occurred at epoch 6673 with best_epoch = 5873 and best_val_0_rmse = 494212.39827


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
2it [55:42, 1696.16s/it]


Early stopping occurred at epoch 7847 with best_epoch = 7047 and best_val_0_rmse = 562527.68416


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
3it [1:26:31, 1765.95s/it]


Early stopping occurred at epoch 7888 with best_epoch = 7088 and best_val_0_rmse = 453840.75203


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
4it [2:00:37, 1876.25s/it]


Early stopping occurred at epoch 8642 with best_epoch = 7842 and best_val_0_rmse = 348376.91993


/usr/local/lib/python3.7/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
5it [2:27:34, 1770.93s/it]


Early stopping occurred at epoch 6912 with best_epoch = 6112 and best_val_0_rmse = 577908.83874


In [ ]:
print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )

The CV score is 487373.31863


In [ ]:
RMSLE = np.sqrt( mean_squared_error(y_true, predictions) )
print("The LB score is %.5f" % RMSLE )

The LB score is 908887.76513
